## Create resstock end-use load dataframes at desired aggregation level
1. Import data (from ResStock Parquet files in SMART-DS)
2. Aggregate at resstock building level or aggregated at smart-ds feeder / regional level (aggregates each building type the number of times it appears in phase loads in the feeder/region)
3. Add cooling sum
4. Save as dictionary 

### Import Packages

In [1]:
import numpy as np
import pyarrow.parquet as pq
import joblib
import scipy
import pandas as pd
import time
import os
import re
import glob
from datetime import datetime, timedelta
import yaml
import pprint

from src import figure_ops
from src import input_ops
from src import model_ops

### Load config file

In [6]:
config_file_name = 'config1'; config_path = f"config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)
pprint.pprint(config, sort_dicts=False)

{'aggregation_level': 'feeder',
 'Y_column': 'cooling_kw_sum',
 'X_columns_set': 'X_columns_D',
 'prediction_model': 'MLP_2_100_10',
 'start_month': 1,
 'end_month': 12,
 'smart_ds_years': ['2018'],
 'TGW_weather_years': ['2058'],
 'TGW_scenario': 'rcp85cooler',
 'CITY_REGIONS_TO_RUN': {'GSO': ['rural']},
 'building_types': ['res', 'com'],
 'input_data_dict_name': 'input_data_dict_proc',
 'input_data_training_path': '/nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/training/2018/months_1_12/ml_input_data/resstock/amy2018/feeder/',
 'output_data_training_path': '/nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/training/2018/months_1_12/ml_output_data/cooling_kw_sum/X_columns_D/feeder/',
 'input_data_prediction_path': '/nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/prediction/input/TGW_weather',
 'output_data_prediction_path': '/nfs/turbo/seas-mtcraig-climate/Aviad/load_prediction/results/data/prediction/output/2018/months_1_12/cool

### Import, process and save input data

In [17]:
# Import, process and save load and weather data (run only once per dataset, e.g., run again if want to add new regions or years)
start_time = time.time()

# # Define dictionary to store results
smartds_load_dict = {}
smart_ds_year = config['smart_ds_years'][0]

smart_ds_load_path = config['smart_ds_load_path'] + f"/{smart_ds_year}"
CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']
aggregation_level = config['aggregation_level']
building_types = config['building_types']
start_month = config['start_month']
end_month = config['end_month']

columns_to_sum = ['cooling_kw', 'fans_kw', 'refrigeration_kw']

# Loop through all city, region, year, and building type combinations
for city, regions in CITY_REGIONS_TO_RUN.items():
    for region in regions:
        ### Import and process smart-ds load data ###
        parquet_data_path = f'/nfs/turbo/seas-mtcraig-climate/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/load_data' # path to parquet files
        region_directory = f"/nfs/turbo/seas-mtcraig-climate/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss/" # path to where feeder folders are
        # Create a list of load_models (regional / feeders / buildings)   
        match aggregation_level:
            case 'regional':
                load_models = ['regional']
            case 'feeder':
                # load_models = find_feeder_folders(region_directory) # list of feeders in the format of '/substation/feeder/'
                load_models = input_ops.make_feeder_list(region_directory) # list of feeders
                print(f'...Loaded feeders in {smart_ds_year} {city} {region}: {load_models}')
            case _: # building
                load_models = sorted([f for f in os.listdir(parquet_data_path) if f.endswith(".parquet")]) # list of buildings in the format of 'building_name.parquet' 
                # print(f'...Loaded buildings in {smart_ds_year} {city} {region}: {load_models}')
        print(f'......Creating data frame for {smart_ds_year} {city} {region} ......')
        for load_model in load_models:
            for building_type in building_types:
                if aggregation_level == 'regional' or aggregation_level == 'feeder':
                    print(f'......Creating data frame for {smart_ds_year} {city} {region} {load_model} {building_type}......')
                    feeder_path_name = input_ops.add_feeder_upper_folder(load_model)
                    feeder_path = f'/nfs/turbo/seas-mtcraig-climate/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss/{feeder_path_name}/Loads.dss'
                    load_df = input_ops.aggregate_parquet_data(parquet_data_path, feeder_path, building_type, aggregation_level, start_month, end_month)  # 'res' for residential | 'com' for commercial
                    if load_df.empty:
                        print(f"Skipped {feeder_path} since it didn't have parquet files found in the given folder with the specified prefix.")
                        continue 
                else:
                    load_prefix = load_model.split('_')[0]
                    if load_prefix not in {'com', 'res'}:
                        continue
                    load_df = input_ops.get_parquet_load_data(parquet_data_path, load_model,start_month,end_month)
                
                # Set index to date_time and remove month which gets strage values (temporary solution) 
                load_df = load_df.reset_index()
                load_df.index = load_df["date_time"] # Make date_time the index
                input_df = load_df 
                input_df = input_df.drop('month', axis=1)
                
                # Add Cooling sum
                # Ensure all columns exist in the dataframe before summing
                if all(col in input_df.columns for col in columns_to_sum):
                    # Check if 'cooling_kw_sum' already exists
                    if 'cooling_kw_sum' not in input_df.columns:
                        input_df['cooling_kw_sum'] = input_df[columns_to_sum].sum(axis=1)
                
                # Store in dictionary with (year, city, region, load_model, building_type) as the key
                smartds_load_dict[(smart_ds_year, city, region, load_model, building_type)] = input_df
        # Save dictionary of current region as joblib files
        if aggregation_level == 'building':
            input_data_region_dir = f'{smart_ds_load_path}/{city}/{region}/'
            if not os.path.exists(input_data_region_dir):
                os.makedirs(input_data_region_dir, exist_ok=True)
            print(f'saving joblib for {city} {region}')
            joblib.dump(smartds_load_dict, os.path.join(input_data_region_dir, f"smartds_load_dict.joblib"))                  
                     
# Save dictionary of all regions as joblib files
if aggregation_level == 'regional' or aggregation_level == 'feeder':
    print('saving joblib for all regions')
    input_data_region_dir = f'{smart_ds_load_path}/all_regions/'
    os.makedirs(input_data_region_dir, exist_ok=True)     # Ensure the directory exists
    joblib.dump(smartds_load_dict, os.path.join(input_data_region_dir, "smartds_load_dict.joblib")) # Save the file

end_time = time.time(); print(f"Runtime for loading data: {(end_time - start_time) / 60:.2f} minutes")

......Creating data frame for 2017 GSO rural ......
......Creating data frame for 2017 GSO rural regional res......
......Creating data frame for 2017 GSO rural regional com......
......Creating data frame for 2017 GSO industrial ......
......Creating data frame for 2017 GSO industrial regional res......
......Creating data frame for 2017 GSO industrial regional com......
......Creating data frame for 2017 GSO urban-suburban ......
......Creating data frame for 2017 GSO urban-suburban regional res......
......Creating data frame for 2017 GSO urban-suburban regional com......
......Creating data frame for 2017 AUS P1R ......
......Creating data frame for 2017 AUS P1R regional res......
......Creating data frame for 2017 AUS P1R regional com......
......Creating data frame for 2017 AUS P1U ......
......Creating data frame for 2017 AUS P1U regional res......
......Creating data frame for 2017 AUS P1U regional com......
......Creating data frame for 2017 AUS P2U ......
......Creating data 

In [18]:
smartds_load_dict.keys()

dict_keys([('2017', 'GSO', 'rural', 'regional', 'res'), ('2017', 'GSO', 'rural', 'regional', 'com'), ('2017', 'GSO', 'industrial', 'regional', 'res'), ('2017', 'GSO', 'industrial', 'regional', 'com'), ('2017', 'GSO', 'urban-suburban', 'regional', 'res'), ('2017', 'GSO', 'urban-suburban', 'regional', 'com'), ('2017', 'AUS', 'P1R', 'regional', 'res'), ('2017', 'AUS', 'P1R', 'regional', 'com'), ('2017', 'AUS', 'P1U', 'regional', 'res'), ('2017', 'AUS', 'P1U', 'regional', 'com'), ('2017', 'AUS', 'P2U', 'regional', 'res'), ('2017', 'AUS', 'P2U', 'regional', 'com'), ('2017', 'SFO', 'P1R', 'regional', 'res'), ('2017', 'SFO', 'P1R', 'regional', 'com'), ('2017', 'SFO', 'P1U', 'regional', 'res'), ('2017', 'SFO', 'P1U', 'regional', 'com'), ('2017', 'SFO', 'P2U', 'regional', 'res'), ('2017', 'SFO', 'P2U', 'regional', 'com')])

In [19]:
smartds_load_dict[('2017', 'GSO', 'rural', 'regional', 'com')]

,index,date_time,building,pf,total_site_electricity_kw,total_site_electricity_kvar,heating_kw,heating_kvar,cooling_kw,cooling_kvar,...,plug_loads_kvar,clothes_dryer_kw,clothes_dryer_kvar,clothes_washer_kw,clothes_washer_kvar,stove_kw,stove_kvar,dishwasher_kw,dishwasher_kvar,cooling_kw_sum
date_time,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,0,2017-01-01 00:00:00,12789,0.999240,1100.150992,190.417184,76.869683,0.139824,20.335409,5.931161,...,91.362393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.289584
2017-01-01 01:00:00,4,2017-01-01 01:00:00,12789,0.999240,1070.287649,198.325677,43.899095,0.188650,17.974978,5.242702,...,90.972396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,119.122399
2017-01-01 02:00:00,8,2017-01-01 02:00:00,12789,0.999241,1059.379258,192.512802,42.562445,0.139824,17.865295,5.210711,...,90.877923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.042853
2017-01-01 03:00:00,12,2017-01-01 03:00:00,12789,0.999517,1093.739552,192.368007,73.839664,0.188650,18.619965,5.430823,...,91.312765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,116.038161
2017-01-01 04:00:00,16,2017-01-01 04:00:00,12789,0.989118,1233.405866,217.200175,161.899167,0.188650,17.961822,5.238865,...,93.076902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,158.121644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:00:00,35020,2017-12-31 19:00:00,12789,0.996207,2270.516234,331.528995,749.228473,1.015645,8.948358,2.609938,...,123.953262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,264.885105
2017-12-31 20:00:00,35024,2017-12-31 20:00:00,12789,0.996172,2159.902948,305.362663,709.757780,1.015645,8.161375,2.380401,...,118.543146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,252.387068
2017-12-31 21:00:00,35028,2017-12-31 21:00:00,12789,0.996313,1918.052376,272.362666,610.337260,0.381891,9.038647,2.636272,...,109.087093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,210.381093
